In [1]:
import time
import sys
sys.path.append('../')

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from helpers.pytorch_helper import *

In [2]:
train = InstacartDataset('training.npy','training_labels.npy', transform=ToTensor())
validation = InstacartDataset('validation.npy','validation_labels.npy', transform=ToTensor())

In [3]:
BATCH = 32
train_loader = DataLoader(train, batch_size=BATCH, 
                          shuffle=True, num_workers=10)
val_loader = DataLoader(validation, batch_size=BATCH,
                        shuffle=True, num_workers=10)

In [4]:
class TesterLSTM(nn.Module):
    def __init__(self):
        super(TesterLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=148, 
                            hidden_size=256,
                            num_layers=3,
                            batch_first=False,
                            bidirectional=False)
        self.fc = nn.Linear(256,145)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (Variable(torch.zeros(3, BATCH, 256)).cuda(), # shape is (num_layers,batch_size,hidden_dim)
                Variable(torch.zeros(3, BATCH, 256)).cuda())
    
    def forward(self, x):
        output, self.hidden = self.lstm(x,self.hidden)
        output = self.fc(output.view(x.size()[1],-1))
        return torch.clamp(output,0,49688)

In [5]:
tester = TesterLSTM().cuda()

In [6]:
LR = .01
EPOCHS = 100
loss_function = nn.MSELoss()
optimizer = torch.optim.RMSprop(tester.parameters(), lr=LR)

In [7]:
for epoch in range(EPOCHS):  # loop over the dataset multiple times
    epoch_loss = 0.
    running_loss = 0.
    time0 = time.time()
    # Training
    j=0
    for i, batch in enumerate(train_loader):
        # get the inputs
        inputs, labels = batch['features'], batch['target']
        # wrap them in Variable
        inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = tester(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.data[0]
        epoch_loss += loss.data[0]
        if i % 1000 == 0:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0
        j+=1
    print('Epoch %d finished. Loss: %.5f' % (epoch+1,epoch_loss/(j+1)))
    print('Epoch %d took %.3f seconds.' % (epoch+1, time.time()-time0))
    
    # Validation
    scratch.eval()
    best_val_loss = float('inf')
    j=0
    for i, batch in enumerate(val_loader):
        inputs, labels = batch['features'], batch['target']
        inputs, labels = Variable(inputs, volatile=True).cuda(), Variable(labels, volatile=True).cuda()
        outputs = scratch(inputs)
        val_loss += loss_function(outputs, labels).data[0]
        # Save checkpoint
        is_best = val_loss < best_val_loss
        best_val_loss = min(val_loss, best_val_loss)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': scratch.state_dict(),
            'best_val_loss': best_val_loss,
            'optimizer' : optim.state_dict(),
        }, is_best)
        j+=1
    print("Validation loss for epoch %d: %.5f" % (epoch+1, val_loss/(i+1)))
print('Finished Training')

torch.cuda.FloatTensor torch.cuda.DoubleTensor


AssertionError: 